## TEXT RANK on Clusters 

To get top ranked relevant sentences in the cluster for in-depth cluster analysis.

In [ ]:
from sentence_transformers import SentenceTransformer
bmodel = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
import pandas as pd

df = pd.read_csv('data/cluster.csv')

In [ ]:
import numpy as np
from tqdm import tqdm
import networkx
import matplotlib.pyplot as plt
%matplotlib inline

def textRank(cluster):
    all_words = ''.join([word for word in df[df['Label'] == cluster]['SpeechNormalize']]).replace('\n', '.')
    df_analysis = pd.DataFrame(all_words.split('.'), columns=['Speech'])
    df_analysis = df_analysis.loc[df_analysis['Speech'].str.split(' ').apply(len) > 8]
    df_analysis.reset_index(inplace=True)
    bert_embeddings = bmodel.encode(df_analysis['Speech'])
    a = bert_embeddings[0]
    b = bert_embeddings[1]
    cos_sim = (a @ b.T) / (np.linalg.norm(a)*np.linalg.norm(b))
    
    M = np.eye(7800)

    for i,a in tqdm(enumerate(bert_embeddings)):
        for j,b in enumerate(bert_embeddings):
            if i != j:
                M[i,j] = (a @ b.T) / (np.linalg.norm(a)*np.linalg.norm(b))
                
    similarity_graph = networkx.from_numpy_array(M)
    return similarity_graph


In [ ]:
similarity_graph = textRank(0)
plt.figure(figsize=(1200, 800))
networkx.draw_networkx(similarity_graph, node_color='blue')

scores=networkx.pagerank(similarity_graph)
nx_ranked_sentences = sorted(((score, index) for index, score 
                                            in scores.items()),reverse=True)